In [1]:
from socket import *
import os
from pathlib import Path
import socket 
from _thread import *
import threading 

In [ ]:
my_lock=threading.Lock()
# thread fuction 
def threaded(c,message):
    print('this message : '+message)
    while True: 
        typeOfRequest = message.split()[0]
        filename = message.split()[1]
        sendingport=message.split()[2]
        if typeOfRequest == 'GET':
            print ('File name is: ', filename)
            my_file = Path(filename)
            if my_file.is_file(): 
                with open(filename, 'r') as f:
                    outputdata = f.read()
                    c.send("HTTP/1.1 200 OK\r\n".encode())
                    for i in range(0, len(outputdata)):
                        print(outputdata[i])
                        c.send(outputdata[i].encode())
                    c.send("\r\n".encode())   
                    c.send("\r\n".encode()) 
                break
            else:
                c.send(("HTTP/1.1 404 file not Found\r\n\r\n").encode())
                break
        elif typeOfRequest == 'POST':
            message =c.recv(1024).decode() 
            if message !="HTTP/1.1 404 file not Found\r\n\r\n":
                with open(filename, 'w+') as f:
                    while message != "\r\n":
                        message =c.recv(1024).decode()
                        print(message)
                        f.write(message)
                ###break
            else:
                print(message)
                break
        else:
            c.send(("HTTP/1.1 400 Bad Request\r\n\r\n").encode())
            my_lock.release()
            #c.close()
            #break
        #c.close()
    c.close() 


def Main(): 
    host = "" 
    port = 12345
    #s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    #s.bind((host, port)) 
    #print("socket binded to post", port) 
    #s.listen(5) 
    #print("socket is listening") 
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    s.bind((host, port)) 
    print("socket binded to post", port) 
    s.listen(5) 
    print("socket is listening") 
    while True: 
        c, addr = s.accept() 
        print("socket binded to post", port) 
        # lock acquired by client 
        my_lock.acquire() 
        message = c.recv(1024).decode()
        print('Connected to :', addr[0], ':', addr[1]) 
        print('message : '+message)
        my_lock.release()
        start_new_thread(threaded, (c,message)) 
        #my_lock.release()
        #c.send('200 OK'.encode('ascii'))
    s.close() 


if __name__ == '__main__': 
    Main() 



socket binded to post 12345
socket is listening
socket binded to post 12345
Connected to : 127.0.0.1 : 37753
message : POST hola.txt 12345
this message : POST hola.txt 12345
socket binded to post 12345
Connected to : 127.0.0.1 : 37756
message : POST hola.txt 12345
this message : POST hola.txt 12345
GET abc.txt 12345


In [ ]:
import psutil
from psutil import process_iter
from signal import SIGTERM # or SIGKILL

for proc in process_iter():
    try:
        for conns in proc.connections(kind='inet'):
            if conns.laddr.port == 12345:
                    proc.send_signal(SIGTERM) # or SIGKILL
    except (psutil.ZombieProcess, psutil.AccessDenied, psutil.NoSuchProcess):   
        continue